<a href="https://colab.research.google.com/github/palxx/cs164_prep/blob/main/AI24Ch3a_participation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
# You can use it for tasks like:
# - Generating random integers: random.randint(a, b)
# - Choosing a random element from a list: random.choice(list)
import heapq
# which are binary trees where each parent node is less than or equal to its children (min heap).
# This is useful for tasks like:
#- Finding the smallest element in a collection efficiently: heapq.nsmallest(n, iterable)

# Tile Sliding Domain: Initial State Space

The code defines a 15-puzzle problem and implements search algorithms (Breadth-First Search) to solve it. Below code takes statedimension where we can have as much big number of tiles as we want, our initial state and goal state, actions that we can take.

In [6]:
StateDimension=4
InitialState =  [1,2,3,4,5,6,7,8,13, 14, 9, 12, 10, 11, 15, 0]
GoalState= [1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13, 14, 15, 0]
Actions = lambda s: ['u', 'd', 'l', 'r']
Opposite=dict([('u','d'),('d','u'),('l','r'),('r','l'), (None, None)])

Below function the function Result(state, action) takes the current state of the puzzle and an action (either 'u', 'd', 'l', or 'r' for up, down, left, right) and returns the new state after applying the action.

In [7]:
def Result(state, action):
  i = state.index(0)
  newState = list(state)
  row,col=i//StateDimension, i % StateDimension
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return newState
  if action=='u':
    l,r = row*StateDimension+col, (row-1)*StateDimension+col
  elif action=='d':
    l,r = row*StateDimension+col, (row+1)*StateDimension+col
  elif action=='l':
    l,r = row*StateDimension+col, row*StateDimension+col-1
  elif action=='r' :
    l,r = row*StateDimension+col, row*StateDimension+col+1
  newState[l], newState[r] = newState[r], newState[l]
  return newState

def PrintState(s):
  for i in range(0,len(s),StateDimension):
    print(s[i:i+StateDimension])

def LegalMove(state, action):
  i = state.index(0)
  row,col=i//StateDimension, i % StateDimension
  newState = state.copy()
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return False
  return True


In [8]:
PrintState(InitialState)

[1, 2, 3, 4]
[5, 6, 7, 8]
[13, 14, 9, 12]
[10, 11, 15, 0]


In [9]:
PrintState(GoalState)

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 0]


In [10]:
PrintState(InitialState)
print()
state1 = Result(InitialState, 'u')
PrintState(state1)
print()
state1 = Result(state1, 'r')
PrintState(state1)

[1, 2, 3, 4]
[5, 6, 7, 8]
[13, 14, 9, 12]
[10, 11, 15, 0]

[1, 2, 3, 4]
[5, 6, 7, 8]
[13, 14, 9, 0]
[10, 11, 15, 12]

[1, 2, 3, 4]
[5, 6, 7, 8]
[13, 14, 9, 0]
[10, 11, 15, 12]


# Random Walk

Take some random moves from a state and return the new state and the sequence of moves.

Do not include moves undoing last move, or having no effect.

In [11]:
def ApplyMoves(actions, state):
  for action in actions:
    state = Result(state, action)
  return state

In [12]:
PrintState(InitialState)
print(['r','r'])
PrintState(ApplyMoves(['r','r'],InitialState))

[1, 2, 3, 4]
[5, 6, 7, 8]
[13, 14, 9, 12]
[10, 11, 15, 0]
['r', 'r']
[1, 2, 3, 4]
[5, 6, 7, 8]
[13, 14, 9, 12]
[10, 11, 15, 0]


In [13]:
def ReverseMoves(actions):
  ret = [Opposite[a] for a in actions]
  ret.reverse()
  return ret

## Problem Class

INITIAL = InitialState  
IsGoal = Goal Test  
Actions = Actions List  
Result = Action Behavior  
ActionCost = Action Cost  

In [14]:
class Problem(object): pass

## Node

The Node class represents a node in the search tree for solving the 15-puzzle. Each node stores information about a particular state of the puzzle.

In [15]:
class Node(object):
  def __init__(self, state, parent=None, action=None, path_cost=0 ):
    self.State=state
    self.Parent=parent
    self.Action=action
    self.PathCost = path_cost

  def __str__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __repr__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __lt__(self, other):
    return self.PathCost < other.PathCost;

## Expand

In [16]:
def Expand(problem, node):
  ret = []
  s = node.State
  for action in problem.Actions(s):
    sPrime = problem.Result(s, action)
    cost =node.PathCost + problem.ActionCost(s,action,sPrime)
    ret.append(Node(sPrime, node, action, cost))
  return ret


## Breadth-First Search

The BreadthFirstSearch(problem) function implements the Breadth-First Search algorithm to find a solution to the 15-puzzle defined by the problem object.

In [17]:
def BreadthFirstSearch(problem):
  node = Node(tuple(problem.INITIAL))
  if problem.IsGoal(node.State):
    return node, 0
  Frontier = []
  Frontier.append(node)
  reached = set()
  reached.add(tuple(problem.INITIAL))
  nodesExpanded = 0
  while (Frontier):
    ### print([str(n) for n in Frontier])
    node = Frontier.pop(0)
    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      ### print (s, "IsGoal=", problem.IsGoal(s))
      if problem.IsGoal(s):
        return child, nodesExpanded
      if s not in reached:
        reached.add(s)
        Frontier.append(child)
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Best-First Search

## Problem 1

This code snippet initializes a Problem instance called TileSliding for the 15-puzzle and then performs some basic tests.
Defines the goal state of the puzzle as a lambda function, checking if the given state s matches the solved configuration. Assigns the previously defined Actions function to the Actions attribute of the TileSliding object, defining the possible actions for the puzzle. Assigns the previously defined Result function to the Result attribute of the TileSliding object, defining how actions affect the puzzle state.

In [32]:
TileSliding = Problem()
TileSliding.INITIAL = InitialState
TileSliding.IsGoal = lambda s: s==(1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13, 14, 15, 0)
TileSliding.Actions = Actions
TileSliding.Result=Result
TileSliding.ActionCost = lambda s, a, sPrime: 1
print( TileSliding.IsGoal((1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13, 14, 15, 0)) )
print( Node(InitialState) )
print(1+TileSliding.ActionCost(1,2,3))

True
[1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 9, 12, 10, 11, 15, 0], <none>
2


In [38]:
TileSliding.INITIAL = [1,2,3,4,5,6,7,8,13, 14, 9, 12, 10, 11, 15, 0]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0], r 7398


In [39]:
def Solution(node):
  if node.Parent==None:
    return []
  return Solution(node.Parent) + [node.Action]


***ret is a Node object representing the goal state found by a search algorithm. It calls the Solution function to extract the sequence of actions that lead from the initial state to the goal state. The result is stored in the variable sol and print the intial and final states and the solution on how we got it.***

In [40]:
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))

['l', 'l', 'u', 'r', 'd', 'l', 'l', 'u', 'r', 'd', 'r', 'r']
[1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 9, 12, 10, 11, 15, 0]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]


In [41]:
TileSliding.INITIAL = [1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13, 14, 15, 0]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0), <none> 0


In [42]:
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))

[]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
